<h1> Project 2 </h1>

- task: plotting gdp data on world map
- how should I make the map world library understand countries names? --> project broken down to 2 parts (2nd part in the last module)
1) create a dictionary that maps Pygal country codes to world bank country names
    - i will need plot countries - dict with keys of plotting library country codes and names as values
    - and gdp countries - dict with country names used in gdp data (we dont care about value)
    - returns tuple containg dict and set
    - dict maps country codes from library to country names from gdp countries
    - set contains the country codes from plot countries that were not found in gdp countries
2) transform gdp data for given year into a form sutiable for a world map plot
    - 3 inputs
    - gdpinfo dict
    - plot countries dict
    - year string
    - goal is to create a dict mapping countries codes of plotting library to names of countries of the gdp data we have
    - then we can pass it to a library, like PyGal, to pass it for visual
    - tuple of dict and 2 sets as output
    - dict maps country codes from plot countries to the log (base 10) of GDP value for that country in that year
    - 1st set contains the country codes from the plot_countries that were not found in the GDP data file
    - 2nd set containts the country codes from plot countries that were found in the data of GDP but not for that year
3) create SVG image of the GDP data plotted on the world map
    - 4 inputs
    - gdpinfo dict
    - plot countries dict
    - year string
    - map file
    - no output, only svg
- next part will be about refinning the visualization, mainly about getting the unmatched data matched

In [5]:
pip install pygal_maps_world

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pygal_maps_world: filename=pygal_maps_world-1.0.2-py3-none-any.whl size=278555 sha256=18eb089c052496e4dc497b22a392f9f55290604a972b4af82e9c176d9f2c764c
  Stored in directory: c:\users\michal\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\5c\36\95\a35ef0871d10bc0967ed33b531d8714d397536a3f091544469
Successfully built pygal_maps_world
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Michal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [113]:
"""
Project for Week 3 of "Python Data Visualization".
Unify data via common country name.

Be sure to read the project description page for further information
about the expected behavior of the program.
"""

import csv
import math
import pygal
from pygal_maps_world.i18n import COUNTRIES 

gdpinfo = {
    "gdpfile": "isp_gdp.csv",        # Name of the GDP CSV file
    "separator": ",",                # Separator character in CSV file
    "quote": '"',                    # Quote character in CSV file
    "min_year": 1960,                # Oldest year of GDP data in CSV file
    "max_year": 2015,                # Latest year of GDP data in CSV file
    "country_name": "Country Name",  # Country name field name
    "country_code": "Country Code"   # Country code field name
}


def read_csv_as_nested_dict(filename, keyfield, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """
    with open(filename, "r") as csv_file:
      csvfile = csv.reader(csv_file, delimiter= separator, quotechar = quote)
      outer_dict = {}       #getting the output dict ready

      #IN THIS PART IM GETTING HEADERS TO BE USED IN THE INNER DICTS CREATIONS
      counter = 0
      for line in csvfile:
        counter = counter + 1
        if counter == 1:
          header_list = line
          break

      #IN THIS PART I NEED TO MAKE THE INNTER DICT CREATION LOOP
      counter = 0
      for line in csvfile:
        counter = counter + 1
        if counter == 1:
          pass             #skipping the first row here
        inner_dict = {}     #making the inner dict for each country
        key_value = line[header_list.index(keyfield)] #key_value (the mapping one) is taken via the index in the header
        for number, column in enumerate(line):          #taking columns of each line and their order
          inner_dict[header_list[number]] = column     #same counts of columns everywhere --> using enum index to make key:value pair
        outer_dict[key_value] = inner_dict              #adding the inner dict to the outer one
    return outer_dict

gdp_countries = read_csv_as_nested_dict(gdpinfo["gdpfile"], gdpinfo["country_name"], gdpinfo["separator"], gdpinfo["quote"])
plot_countries = COUNTRIES

def reconcile_countries_by_name(plot_countries, gdp_countries):
    """
    Inputs:
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      gdp_countries  - Dictionary whose keys are country names used in GDP data

    Output:
      A tuple containing a dictionary and a set.  The dictionary maps
      country codes from plot_countries to country names from
      gdp_countries The set contains the country codes from
      plot_countries that were not found in gdp_countries.
    """
    result_dict = {}
    result_set = set()
    for plot_country in plot_countries:
        if plot_countries[plot_country] in gdp_countries.keys():    #comparing names against names
            result_dict[plot_country] = plot_countries[plot_country]#using plot_countries data bcs they match here
        else:
            result_set.add(plot_country)
    return result_dict, result_set

#result_dict, result_set = reconcile_countries_by_name(plot_countries, gdp_countries)


def build_map_dict_by_name(gdpinfo, plot_countries, year):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for

    Output:
      A tuple containing a dictionary and two sets.  The dictionary
      maps country codes from plot_countries to the log (base 10) of
      the GDP value for that country in the specified year.  The first
      set contains the country codes from plot_countries that were not
      found in the GDP data file.  The second set contains the country
      codes from plot_countries that were found in the GDP data file, but
      have no GDP data for the specified year.
    """
    result_dict = {}
    gdp_countries = read_csv_as_nested_dict(gdpinfo["gdpfile"], gdpinfo["country_name"], gdpinfo["separator"], gdpinfo["quote"])
    source_dict, result_set1 = reconcile_countries_by_name(plot_countries, gdp_countries)
    result_set2 = set()
    for country in source_dict:
        if gdp_countries[source_dict[country]][str(year)] == "":
            # source_dict[country] -> country name -> adding [YEAR] -> returns GDP for the year
            result_set2.add(country)
        else:
            result_dict[country] = math.log10(float(gdp_countries[source_dict[country]][str(year)]))
    return result_dict, result_set1, result_set2

# result_dict, result_set1, result_set2 = build_map_dict_by_name(gdpinfo, plot_countries, 2000)
# print(result_dict)

# worldmap_chart = pygal.maps.world.World()
# worldmap_chart.title = 'Some countries'
# worldmap_chart.add("Found", result_dict)
# worldmap_chart.add("NOT found", result_set1)
# worldmap_chart.add("NULL value", result_set2)
# worldmap_chart.render_to_file("test_map_chart.svg")

def render_world_map(gdpinfo, plot_countries, year, map_file):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for
      map_file       - Name of output file to create

    Output:
      Returns None.

    Action:
      Creates a world map plot of the GDP data for the given year and
      writes it to a file named by map_file.
    """
    result_dict, result_set1, result_set2 = build_map_dict_by_name(gdpinfo, plot_countries, year)
    worldmap_chart = pygal.maps.world.World()
    worldmap_chart.title = f"World map of national GDPs for {year}"
    worldmap_chart.add("Found", result_dict)
    worldmap_chart.add("NOT found", result_set1)
    worldmap_chart.add("NULL value", result_set2)
    worldmap_chart.render_to_file(map_file)

render_world_map(gdpinfo, plot_countries, 2000, "test_world_map_2000.svg")

def test_render_world_map():
    """
    Test the project code for several years.
    """
    gdpinfo = {
        "gdpfile": "isp_gdp.csv",
        "separator": ",",
        "quote": '"',
        "min_year": 1960,
        "max_year": 2015,
        "country_name": "Country Name",
        "country_code": "Country Code"
    }

    # Get pygal country code map
    pygal_countries = pygal.maps.world.COUNTRIES

    # 1960
    render_world_map(gdpinfo, pygal_countries, "1960", "isp_gdp_world_name_1960.svg")

    # 1980
    render_world_map(gdpinfo, pygal_countries, "1980", "isp_gdp_world_name_1980.svg")

    # 2000
    render_world_map(gdpinfo, pygal_countries, "2000", "isp_gdp_world_name_2000.svg")

    # 2010
    render_world_map(gdpinfo, pygal_countries, "2010", "isp_gdp_world_name_2010.svg")

test_render_world_map()